# YOLOFace

Uses code from:
* https://github.com/sthanhng/yoloface
* https://github.com/fyr91/face_detection/blob/master/detect_yolov3.py

Useful: 
* https://towardsdatascience.com/real-time-face-recognition-with-cpu-983d35cc3ec5
* https://github.com/fyr91/face_detection
* https://github.com/sthanhng/yoloface
* https://github.com/dannyblueliu/YOLO-Face-detection
* https://www.baseapp.com/deepsight-image-recognition-sdk/deepsight-face-sdk-download/

In [1]:
import cv2
import os
import numpy as np
import glob
import datetime

### Parameters

In [2]:
CONF_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4
IMG_WIDTH = 416
IMG_HEIGHT = 416

### Helper Functions

In [3]:
# Get the names of the output layers
def get_outputs_names(net):
    # Get the names of all the layers in the network
    layers_names = net.getLayerNames()

    # Get the names of the output layers, i.e. the layers with unconnected
    # outputs
    return [layers_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


# Draw the predicted bounding box
def draw_predict(frame, conf, left, top, right, bottom):
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

    text = '{:.2f}'.format(conf)

    # Display the label at the top of the bounding box
    label_size, base_line = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

    top = max(top, label_size[1])
    cv2.putText(frame, text, (left, top - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                (255, 255, 255), 1)
    
    # Add the blurring in
    roi = frame[top : bottom, left : right]

    # Blur the coloured image
    blur = cv2.GaussianBlur(roi, (101, 101), 0)

    # Insert the blurred section back into image
    frame[top : bottom, left : right] = blur


def post_process(frame, outs, conf_threshold, nms_threshold):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.
    confidences = []
    boxes = []
    final_boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant
    # overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold,
                               nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        final_boxes.append(box)
        left, top, right, bottom = refined_box(left, top, width, height)
        # draw_predict(frame, confidences[i], left, top, left + width,
        #              top + height)
        draw_predict(frame, confidences[i], left, top, right, bottom)
    return final_boxes


def refined_box(left, top, width, height):
    right = left + width
    bottom = top + height

    original_vert_height = bottom - top
    top = int(top + original_vert_height * 0.15)
    bottom = int(bottom - original_vert_height * 0.05)

    margin = ((bottom - top) - (right - left)) // 2
    left = left - margin if (bottom - top - right + left) % 2 == 0 else left - margin - 1

    right = right + margin

    return left, top, right, bottom

def blurFacesYOLO(frame, CONF_THRESHOLD):
      
    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT),[0, 0, 0], 1, crop=False)
    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(get_outputs_names(net))
    
    # Remove the bounding boxes with low confidence
    faces = post_process(frame, outs, CONF_THRESHOLD, NMS_THRESHOLD)
    
    return frame.astype(np.uint8)

def blursingle(image, CONF_THRESHOLD):
    cv2.imwrite(os.path.splitext(image)[0] + "_blurred.jpg",
               blurFacesYOLO(cv2.imread(image), CONF_THRESHOLD))

def blurfolder(folder, CONF_THRESHOLD):
    files = glob.glob(os.path.join(folder,"*.jpg"))
    for file in files:
        blursingle(file, CONF_THRESHOLD)
        
def blurvideo(video, CONF_THRESHOLD):
    # Get first frame
    cap = cv2.VideoCapture(video)
    
    
    video_writer = cv2.VideoWriter(os.path.join(os.path.splitext(video)[0] + "_blurred.avi"),
                                       cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),
                                       cap.get(cv2.CAP_PROP_FPS), (
                                           round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                           round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
    # While there are still frames to process...
    while True:
        has_frame, frame = cap.read()

        # Stop the program if reached end of video
        if not has_frame:
            break
            
        video_writer.write(blurFacesYOLO(frame,CONF_THRESHOLD))

### Load Model

In [15]:
# Load the model config
net = cv2.dnn.readNetFromDarknet(os.path.abspath("/Users/tasanders/Documents/weights/yolov3-face.cfg"),
                             os.path.abspath("/Users/tasanders/Documents/weights/yolov3-wider_16000.weights")) # Weights from yoloface github page

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)

### Run on Single File

In [9]:
path_to_image = "/Users/tasanders/Documents/Face_blurring/CC4B73632D38_2021_0210_144434_434_0001.jpg"
blursingle(path_to_image, 0.25)

### Run on folder

In [16]:
%%time
path_to_folder = "/Users/tasanders/Documents/face_test"
blurfolder(path_to_folder,0.25)

CPU times: user 1min 11s, sys: 1.47 s, total: 1min 12s
Wall time: 8.51 s


### Run on video

In [ ]:
path_to_video = "/path_to_video/video.mp4"
blurvideo(path_to_video, 0.25)